In [21]:
from collections import deque
try:
    while True:
        n, m = map(int, input().split())
        board = [list(map(int, input().split())) for _ in range(n)]
        dx = [0,1,1]
        dy = [1,1,0]

        res = 0
        for i in range(n):
            for j in range(m):
                if board[i][j] == 1:
                    q = deque()
                    q.append((i,j))
                    tmp = 0
                    ch = [[False]*m for _ in range(n)]
                    while q:
                        flag = False
                        for _ in range(len(q)):
                            x, y = q.popleft()
                            for k in range(3):
                                xx = x + dx[k]
                                yy = y + dy[k]
                                if 0<=xx<n and 0<=yy<m and not ch[xx][yy] and board[xx][yy] == 1:
                                    ch[xx][yy] = True
                                    q.append((xx,yy))
                                elif not(0<=xx<n and 0<=yy<m) or board[xx][yy] == 0 :
                                    flag = True
                                    break
                        if flag:
                            break
                        tmp += 1
                    
                    if tmp >= 1:
                        res = max(res, tmp+1)
        print(res)
except:
    pass

3


In [ ]:
while True:
    n, m = map(int, input().split())
    if n == 0 and m == 0:
        break
    board = [list(map(int, input().split())) for _ in range(n)]
    dp = [[0]*(m+1) for _ in range(n+1)]

    res = 0
    for i in range(1, n+1):
        for j in range(1, m+1):
            if board[i-1][j-1] == 1:
                dp[i][j] = min(dp[i-1][j], dp[i-1][j-1], dp[i][j-1]) + 1
                
                if dp[i][j] > res:
                    res = dp[i][j]
    print(res)